In [11]:
from bs4 import BeautifulSoup
import urllib.request 
import http.cookiejar
ccj = http.cookiejar.CookieJar()

def get_camel_info(asin, headers=None):
    url = 'http://camelcamelcamel.com/product/%s' % asin
    chart_url = 'http://charts.camelcamelcamel.com/us/%s/amazon.png?force=1&zero=0&w=1024&h=640&desired=false&legend=1&ilt=1&tp=all&fo=0&lang=en' % asin
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(ccj))
    
    if headers != None: opener.addheaders = headers
    try:
        content = opener.open(url).read()
    except Exception as e:
        return None
    
    soup = BeautifulSoup(content, "lxml")
    seller_types = [
        {
            'header': 'Amazon Price History'
        },
        {
            'header': '3rd Party New Price History'
        },
        {
            'header': '3rd Party Used Price History'
        }           
    ]        
    data = {
        'prices': {
             'current': {
            },
            'lowest': {
            },
            'highest': {
            
            },
            'average': {
            }
        },
        'url': url,
        'chart_url': chart_url
       
    }
    for seller_type in seller_types:
        header = soup.find('h3', string = seller_type['header'])
        if header == None:
            continue
        type_div = header.parent
        for price_type in data['prices']:
            price_div = type_div.find((lambda tag: tag.name == "td" and price_type.capitalize() in tag.text))
            # print price_div
            try:
                price = price_div.find_next_sibling('td').text.replace('$', '')
                price = float(price)
                date = price_div.find_next_sibling('td').find_next_sibling('td').text
    
            except Exception as e:
                price = None
                date = None
           
            data['prices'][price_type][seller_type['header']] = {
                'price': price,
                'date': date
            }
    #sleep(3)
    return data

get_camel_info('B005ADY8V4')

{'prices': {'current': {'Amazon Price History': {'price': 167.37,
    'date': 'Dec 11, 2018'},
   '3rd Party New Price History': {'price': 155.23, 'date': 'Dec 11, 2018'},
   '3rd Party Used Price History': {'price': None, 'date': None}},
  'lowest': {'Amazon Price History': {'price': 141.24, 'date': 'Oct 06, 2013'},
   '3rd Party New Price History': {'price': 94.9, 'date': 'Jul 04, 2017'},
   '3rd Party Used Price History': {'price': 94.62, 'date': 'Jul 10, 2014'}},
  'highest': {'Amazon Price History': {'price': 189.97,
    'date': 'Feb 05, 2016'},
   '3rd Party New Price History': {'price': 174.08, 'date': 'Feb 05, 2016'},
   '3rd Party Used Price History': {'price': 635.52, 'date': 'Sep 09, 2017'}},
  'average': {'Amazon Price History': {'price': 163.96, 'date': '-'},
   '3rd Party New Price History': {'price': 160.12, 'date': '-'},
   '3rd Party Used Price History': {'price': 452.97, 'date': '-'}}},
 'url': 'http://camelcamelcamel.com/product/B005ADY8V4',
 'chart_url': 'http://cha